# RAG System

![rag](./images/rag1.png)

In [1]:
import tiktoken
from langchain_openai import OpenAIEmbeddings
import numpy as np
import bs4
from langchain_community.document_loaders import WebBaseLoader

import os
from dotenv import load_dotenv

load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

open_api_key = os.getenv("OPEN_API_KEY")

os.environ['OPENAI_API_KEY'] = open_api_key
os.environ['USER_AGENT'] = os.getenv("USER_AGENT")
USER_AGENT = os.getenv("USER_AGENT")

Let's try querying LLM without context

In [2]:
import openai

client = openai.OpenAI(api_key=open_api_key)


def get_completion(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7  # Adjust creativity
        )
    return response.choices[0].message.content

prompt = "Хто такий Ігор Михайлович Черевко?"
response = get_completion(prompt)
print(response)

На жаль, я не можу знайти інформацію про конкретну особу з ім'ям Ігор Михайлович Черевко. Можливо, ця особа не є відомою або вона не є публічно відомою фігурою. Якщо у вас є конкретні запитання або контекст, будь ласка, надайте більше інформації, і я намагатимусь допомогти знайти відповідь.


## What is RAG system?

A RAG (Retrieval-Augmented Generation) system is an AI approach that combines information retrieval with generative AI to improve response accuracy and relevance. It works by first retrieving relevant documents or knowledge from a database and then using a generative model (like GPT) to process and summarize the information.

In [3]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [4]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [7]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
print(len(query_result))
print(query_result[:20])

1536
[-0.0035892894957214594, -0.0030708718113601208, -0.012454708106815815, -0.024972829967737198, -0.028308460488915443, 0.01755327545106411, -0.01554935984313488, -0.008535659871995449, -0.006747357081621885, -0.00487027270719409, -0.003703436581417918, 0.005456861574202776, -0.008351756259799004, -0.001444276887923479, 0.009429811500012875, 0.01955718919634819, 0.03076896257698536, 0.01491521019488573, 0.01570155657827854, -0.015473262406885624]


In [ ]:
print(query_result)

In [9]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8806915835035412


In [10]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://mathmod.chnu.edu.ua/", "https://mathmod.chnu.edu.ua/pro-nas/spivrobitnyky/cherevko-ihor-mykhailovych/")
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header")
    #     )
    # ),
)
blog_docs = loader.load()

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

In [11]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)
print(splits)

# splits = blog_docs.split(' ')
# print(splits)

[Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, page_content='Кафедра математичного моделювання - Кафедра математичного моделювання\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nПерейти до основного вмісту\n\n\n\n\n\n\n[email\xa0protected]\n\n\n\r\n                 вул. Університетська 28, каб. 17\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nНовини\n\nПро нас'), Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, pag

In [19]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(splits, OpenAIEmbeddings())

vectorstore.save_local("../models/")

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [15]:
docs = retriever.get_relevant_documents("Хто такий Ігор Михайлович Черевко?")
print(docs)

[Document(id='69a52421-3950-42ba-95fb-0604de893450', metadata={'source': 'https://mathmod.chnu.edu.ua/pro-nas/spivrobitnyky/cherevko-ihor-mykhailovych/', 'title': 'Черевко Ігор Михайлович - Кафедра математичного моделювання', 'description': 'сторінка Черевка Ігоря Михайловича', 'language': 'uk'}, page_content='>\n\n\nЧеревко Ігор Михайлович\n\n\n\n\n\n\n\n\n\n\n\n\n\n[email\xa0protected]\n\n\n\n\n\n\nЧеревко Ігор Михайлович\nДоктор фізико-математичних наук, професор\nЗавідувач кафедри\nGoogle Scholar\nORCID ID: 0000-0002-2690-2091\nResearcher ID: G-3796-2017 \nSCOPUS Author ID: 15520902400'), Document(id='77dd1da0-b8e8-488f-a874-e66e9ae8b136', metadata={'source': 'https://mathmod.chnu.edu.ua/pro-nas/spivrobitnyky/cherevko-ihor-mykhailovych/', 'title': 'Черевко Ігор Михайлович - Кафедра математичного моделювання', 'description': 'сторінка Черевка Ігоря Михайловича', 'language': 'uk'}, page_content='Ігор Михайлович Черевко розпочав трудову діяльність після закінчення у 1978 році Чернівец

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# also look up -> rag fusion, rag iterative prompting etc.

prompt = ChatPromptTemplate.from_template(template)

In [17]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Хто такий Ігор Михайлович Черевко?")

'Ігор Михайлович Черевко - доктор фізико-математичних наук, професор, завідувач кафедри математичного моделювання.'